## 条件分岐

In [1]:
from z3 import Solver
from meyer.util.z3py_set import set, sets, included, Cpl, Inter
from meyer.program import U, prog, progs, conclude
from meyer.equivalence import eq, equal, eq_set, eq_pre, eq_post
from meyer.feasibility import feasible
from meyer.basic_constructs import Rest
from meyer.special_programs import Fail
from meyer.refinement import is_ref_of
from meyer.conditionals import GCond, Ite

s = Solver()

In [2]:
title = "P44 The guarded conditional is commutative"
C1, C2 = sets('C1 C2', U)
p1, p2 = progs(s, 'p1 p2')
lhs = GCond(C1, p1, C2, p2)
rhs = GCond(C2, p2, C1, p1)
conclude(s, eq(lhs, rhs), title)

P44 The guarded conditional is commutative
Universe = U, has 3 element(s)
Holds: unsat 



In [3]:
title = "P45 both forms of conditional are associative"
# What associative means now?
# a?(b?c) = (a?b)?c
# C1, C2, C3 = sets('C1 C2 C3', U)
# p1, p2, p3 = progs(s, 'p1 p2 p3')

In [4]:
title = "P46 both forms of conditional distribute over choice and concurrency"
# What distribute means now?
# i.e. Restriction distributes over choice C:(p1∪p2) = (C:p1)∪(C:p2)
# C1, C2, C3 = sets('C1 C2 C3', U)
# p1, p2, p3 = progs(s, 'p1 p2 p3')

なかなか何やってもUnknownしか出ない．制限演算の新しい制約を使ってみるか

In [5]:
title = "P47 If D1⊆C1 and D2 ⊆ C2, then (if D1:p [] D2:q end) ⊆ (if C1:p [] C2:q end)"
C1, C2, D1, D2 = sets('C1 C2 D1 D2', U)
p, q = progs(s, "p q")
s.add(feasible(p, q)) # counter example
s.add(included(D1, C1), included(D2, C2))
conclude(s, is_ref_of(GCond(D1, p, D2, q), GCond(C1, p, C2, q)), title)

P47 If D1⊆C1 and D2 ⊆ C2, then (if D1:p [] D2:q end) ⊆ (if C1:p [] C2:q end)
Universe = U, has 3 element(s)
Unholds: unknown
smt tactic failed to show goal to be sat/unsat (incomplete quantifiers) 



In [6]:
title = "If q1⊆p1 and q2 ⊆ p2, then (if C:q1 [] C:q2 end) ⊆ (if C:p1 [] C:p2 end)"
C = set('C', U)
p1, p2, q1, q2 = progs(s, "p1 p2 q1 q2")
s.add(feasible(p1, p2, q1, q2)) # counter example
s.add(is_ref_of(q1, p1), is_ref_of(q2, p2))
conclude(s, is_ref_of(GCond(C, q1, C, q2), GCond(C, p1, C, p2)), title)

If q1⊆p1 and q2 ⊆ p2, then (if C:q1 [] C:q2 end) ⊆ (if C:p1 [] C:p2 end)
Universe = U, has 3 element(s)
Unholds: sat
content of C_36
 = True

set of p2_41
 else -> False

pre of p2_41
 else -> False

post of p2_41
 else ->
	 if -> Var(0) == C
	 then ->
		 else -> False
	 else ->
		 else -> False

set of q1_45
 else -> True

pre of q1_45
 else -> Not(And(Not(Var(0) == C), Var(0) == A))

post of q1_45
 else ->
	 if -> Var(0) == C
	 then ->
		 B -> False
		 else -> True
	 else ->
		 else -> And(Not(Var(0) == C), Var(0) == A)

set of p1_37
 else -> Not(Var(0) == C)

pre of p1_37
 else -> And(Not(Var(0) == C), Not(Var(0) == A))

post of p1_37
 else ->
	 if -> Var(0) == C
	 then ->
		 else -> False
	 else ->
		 else -> Not(Var(0) == C)

set of q2_49
 else -> True

pre of q2_49
 else -> Not(Var(0) == C)

post of q2_49
 else ->
	 if -> Var(0) == C
	 then ->
		 else -> True
	 else ->
		 if -> And(Not(Var(0) == C), Not(Var(0) == A))
		 then ->
			 else -> True
		 else ->
			 else -> True



In [7]:
title = "P49 If q1⊆p1 and q2 ⊆ p2, then (if C then q1 else q2 end) ⊆ (if C then p1 else p2 end)"
C = set('C', U)
p1, p2, q1, q2 = progs(s, "p1 p2 q1 q2")
s.add(is_ref_of(q1, p1), is_ref_of(q2, p2))
conclude(s, is_ref_of(Ite(C, q1, q2), Ite(C, p1, p2)), title)

P49 If q1⊆p1 and q2 ⊆ p2, then (if C then q1 else q2 end) ⊆ (if C then p1 else p2 end)
Universe = U, has 3 element(s)
Holds: unsat 



In [8]:
title = "P50 (if C then p1 else p2 end) = (if C' then p2 else p1 end)"
C = set('C', U)
p1, p2 = progs(s, 'p1 p2')
conclude(s, eq(Ite(C, p1, p2), Ite(Cpl(C), p2, p1)), title)

P50 (if C then p1 else p2 end) = (if C' then p2 else p1 end)
Universe = U, has 3 element(s)
Holds: unsat 



In [9]:
title = "P51 (C:p) = (if C:p end) (= if C:p [] D:Fail)"
C, D = sets('C D', U)
p = prog(s, 'p')
conclude(s, eq(Rest(C, p), GCond(C, p, C, Fail())), title)

P51 (C:p) = (if C:p end) (= if C:p [] D:Fail)
Universe = U, has 3 element(s)
Holds: unsat 



変数を二つ使う方でむしろCounterExampleを出せる

In [10]:
title = "P52 (if C1:p1 [] C2:p2 end) ⊆ C1:p1"
C1, C2 = sets('C1 C2', U)
p1, p2 = progs(s, 'p1 p2')
s.add(feasible(p1, p2))
conclude(s, is_ref_of(GCond(C1, p2, C2, p2), Rest(C1, p1)), title)

P52 (if C1:p1 [] C2:p2 end) ⊆ C1:p1
Universe = U, has 3 element(s)
Unholds: unknown
smt tactic failed to show goal to be sat/unsat (incomplete quantifiers) 



In [11]:
title = "P53 (D:(if C1:p [] C2:q end)) = (if (D∩C1):p [] (D∩C2):q end)"
C1, C2, D = sets('C1 C2 D', U)
p, q = progs(s, "p q")
lhs = Rest(D, GCond(C1, p, C2, q))
rhs = GCond(Inter(D, C1), p, Inter(D, C2), q)
conclude(s, eq(lhs, rhs), title)

P53 (D:(if C1:p [] C2:q end)) = (if (D∩C1):p [] (D∩C2):q end)
Universe = U, has 3 element(s)
Holds: unsat 



In [12]:
title = "P54 (if C then p1 else p2 end) = (if C:p1 [] C':p2 end)"
C = set('C', U)
p1, p2 = progs(s, 'p1 p2')
conclude(s, eq(Ite(C, p1, p2), GCond(C, p1, Cpl(C), p2)), title)

P54 (if C then p1 else p2 end) = (if C:p1 [] C':p2 end)
Universe = U, has 3 element(s)
Holds: unsat 



In [13]:
title = "P55 (if C then p1 else p2 end) = (if C' then p2 else p1 end)"
# totally same as P50!